# 신경망 학습
**학습** : 훈련 데이터로부터 가중치 매개변수의 최적값   
[손실함수] : 신경망 모델의 예측 값과 실제 값 간의 차이를 측정하는 함수

## 1. 오차 제곱 합
sum of squares for error (SSE)
* 원핫 인코딩을 함.

$$ E = \frac{1}{2} \displaystyle\sum_k(y_k-t_k)^2$$

In [1]:
import numpy as np
def sum_squares_error(y,t):
    return 0.5*np.sum((y-t)**2)

## 2. Cross Entropy Error(CEE)   
교차 엔트로피 오차

$$E = -\displaystyle\sum_k\;t_k\;\text{log}y_k$$

In [ ]:
import numpy as np
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

delta를 더하는 이유는 np.log()함수엥 0을 입력하면 마이너스 무한대.
따라서 아주 작은 값을 더해서 0이 될 수 없게 했음.

## 3. 미니배치
$$ E= -\frac {1}{N} \displaystyle\sum_n\displaystyle\sum_k\; t_{nk}\; \text{log}\;y_{nk} $$

일부만 골라서 학습을 수행하는것.  
**미니배치**


In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.dirname(__file__),'..'))
import numpy as np
from dataset.mnist import loasd_mnist

(x_train,t_train),(x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)

print(x_train.shape) # (60000,784)
print(t_train.shape) # (60000,10)

호출할 때 one_hot_label=True로 지정하여 원핫인코딩으로,   
앞의 코드에서 MNIST 데이터를 읽은 결과,    
훈련데이터는 60,000개고 입력 데이터는 784열 (원래는 28x28)인 이미지 데이터임을 할 수 있다.     
정답레이블은 10줄짜리 데이터임.

### 왜 손실 함수를 설정하는가?
신경망을 학습할 때 정확도를 지표로 삼아서는 안 된다.  
정확도를 지표로 하면 매개변수의 미분이 대부분의 장소에서 0이 되기 때문이다.

## 4. 미분
해석적미분   
$\frac{df(x)}{dx}=\displaystyle\lim_{h\rarr\infin}{\frac{f(x+h)-f(x)}{h}}$
-> 반올림 오차 문제 일으킴   
수치미분 numerical differentiation   
**중심차분** **중앙차분**으로 해결   
$\frac{df(x)}{dx}=\displaystyle\lim_{h\rarr\infin}{\frac{f(x+h)-f(x-h)}{2h}}$

In [4]:
def numerical_diff(f,x):
    h=1e-4 # 0.0001
    return(f(x+h)-f(x-h)/(2*h))

### 편미분

$f(x_0,x_1)\;=\;x^2_0+x^2_1$

In [ ]:
def function_2(x):
    return x[0]**2+x[1]**2
#또는 return np.sum(x**2)

문제: $x_0 = 3, x_1 = 4$ 일때, $x_0$ 에 대한 편미분 $\frac{\partial{f}}{\partial{x_0}}$ 를 구하라.

In [7]:
def function_tmp1(x0):
    return x0*x0 +4.0**2**0

numerical_diff(function_tmp1,3.0)

-64983.99944998999

## 5. 기울기

기울기가 가르키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향

In [8]:
def numerical_gradient(f,x):
    h = 1e-4 #0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val #값 복원
    return grad

In [ ]:
#학습률
def gradient_descent(f,init_x,lr=0.01,step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr*grad
    return x
# 인수 f는 최적화 하려는 함수, init_x는 초깃값, lr은 learning rate를 의미. step_num은 경사법에 따른 반복 횟수
# 학습률을 곱한 값으로 갱신하는 처리를 step_num번 반복


신경망에서도 기울기를 구해야함.   
가중치 매개변수에 대한 손실함수에 대한 기울기.
### simpleNet

In [ ]:
import numpu as np
import sys,os
sys.path.append(os.path.join(os.path.dirname(__file__),'..'))
from common.function import softmax, cross_entropy_error
from common.gradient import numerical_gradient
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) #정규분포로 초기화
        
    def predict(self,x):
        return np.dot(x,self.W)
    def loss(self,x,t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y,t)
        return loss

common/functions.py에 정의한 softmax와 cross_entropy_error 매서드를 의용함   
그리고 common.gradient.py에 정의한 numerical_gradient메서드도 이용.  

simpleNet 클래스는 형상이 2x3인 2차원 배열    

dW의 내용을 보면, $\frac{\partial L}{\partial W}$ 의 $\frac{\partial L}{\partial w_{11}}$ 은 대략 0.2 이다.  
이는   $w_{11}$ 을 $h$ 만큼 늘리면 손실 함수의 값은 $0.2h$ 만큼 증가함을 의미.   

마찬가지로 $\frac{\partial L}{\partial w_{23}}$ 은 대략 -0.5이니,  $w_{23}$ 을 $h$ 만큼 늘리면 손실 함수의 값은 $0.5h$ 만큼 감소함을 의미.   

$\therefore w_{23} 이 w_{11} 보다 더 크게 기여함$ 

### 학습 알고리즘
전체
* 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향ㅇ르 훈련 데이터에 적응하도록 조정하는 과정을 학습이라함.   
* 신경망 학습은 4단계로 수행
1. 미니배치   
* 훈련데이터 중 일부를 무작위로 가져옴.
* 선별된 데이터를 미니배치라 하며, 미니배치의 손실 함수 값을 줄이는 거시 목표
2. 기울기 산출
* 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함.
* 기울기는 손실 함수의 값을 가장 작게 하는 방향
3. 매개변수 갱신
* 가중치 매개변수를 기울기 방향으로 아주 조금 갱신함.
4. 반복
* 1~3단계 반복

이 때, 데이터를 미니배치로 무작위로 선정하기 때문에 경사하강법은 확률적 경사하강법 SGD 라고 부름   

경사법은 기울기를 이용하는거임.

# 4장 신경망 학습 - 시험 대비 핵심 정리

## 1. 학습의 개념 ⭐⭐⭐

### 학습이란?

**훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 찾는 과정**

```python
"""
퍼셉트론: 가중치를 수동으로 설정
신경망: 가중치를 데이터로부터 학습

학습 = 손실 함수를 최소화하는 가중치 찾기
"""
```

### 손실 함수 (Loss Function)

**신경망 모델의 예측 값과 실제 값 간의 차이를 측정하는 함수**

- 신경망 성능의 "나쁨"을 나타내는 지표
- 값이 작을수록 좋은 모델
- 학습의 목표: 손실 함수를 최소화

---

## 2. 손실 함수의 종류 ⭐⭐⭐

### 2.1 오차제곱합 (SSE: Sum of Squares for Error)

**수식:**
$$E = \frac{1}{2}\sum_{k}(y_k - t_k)^2$$

- $y_k$: 신경망의 출력 (예측값)
- $t_k$: 정답 레이블 (원-핫 인코딩)
- $k$: 데이터의 차원 수

**구현:**
```python
import numpy as np

def sum_squares_error(y, t):
    """오차제곱합"""
    return 0.5 * np.sum((y - t)**2)

# 예시
# 정답: 2
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])  # 원-핫 인코딩

# 예측 1: '2'일 확률이 가장 높음 (정답)
y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print(sum_squares_error(y1, t))  # 0.09750000000000003

# 예측 2: '7'일 확률이 가장 높음 (오답)
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
print(sum_squares_error(y2, t))  # 0.5975
```

**특징:**
- 회귀 문제에 주로 사용
- 오차의 제곱을 사용 (큰 오차에 더 큰 페널티)

---

### 2.2 교차 엔트로피 오차 (CEE: Cross Entropy Error) ⭐⭐⭐

**수식:**
$$E = -\sum_{k}t_k\log y_k$$

- 정답일 때의 출력만 영향 (원-핫 인코딩)
- 정답에 해당하는 출력이 클수록 손실 작음

**구현:**
```python
def cross_entropy_error(y, t):
    """교차 엔트로피 오차"""
    delta = 1e-7  # 아주 작은 값
    return -np.sum(t * np.log(y + delta))

# 예시
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

# 예측 1: '2'일 확률이 높음 (정답)
y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print(cross_entropy_error(y1, t))  # 0.510825457099338

# 예측 2: '7'일 확률이 높음 (오답)
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
print(cross_entropy_error(y2, t))  # 2.302584092994546
```

**delta를 더하는 이유:**
```python
# np.log(0) = -∞ (마이너스 무한대)
# 계산 불가능하므로 아주 작은 값을 더함

y = 0
print(np.log(y))        # 오류 또는 -inf
print(np.log(y + 1e-7)) # -16.11809565095832
```

**특징:**
- 분류 문제에 주로 사용
- 정답 레이블에 해당하는 출력의 로그를 계산
- 확률 분포 간의 차이를 측정

---

## 3. 미니배치 학습 ⭐⭐⭐

### 배치용 손실 함수

**수식:**
$$E = -\frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log y_{nk}$$

- $N$: 배치 크기
- 데이터 1개당 평균 손실 함수 계산

### 미니배치란?

**훈련 데이터 중 일부를 무작위로 선택하여 학습하는 방법**

```python
import sys, os
sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dataset.mnist import load_mnist

# 데이터 로드
(x_train, t_train), (x_test, t_test) = load_mnist(
    normalize=True, 
    one_hot_label=True
)

print(x_train.shape)  # (60000, 784)
print(t_train.shape)  # (60000, 10)

# 미니배치 선택
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

print(x_batch.shape)  # (10, 784)
print(t_batch.shape)  # (10, 10)
```

### 배치용 교차 엔트로피 구현

```python
def cross_entropy_error(y, t):
    """
    배치 처리를 지원하는 교차 엔트로피
    
    y: 신경망 출력
    t: 정답 레이블
    """
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    # 배치 크기
    batch_size = y.shape[0]
    
    # 원-핫 인코딩인 경우
    if t.size == y.size:
        return -np.sum(t * np.log(y + 1e-7)) / batch_size
    
    # 정답 레이블이 숫자인 경우
    else:
        return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
```

**사용 예:**
```python
# 원-핫 인코딩
t = np.array([[0, 0, 1, 0],
              [0, 1, 0, 0]])
y = np.array([[0.1, 0.2, 0.6, 0.1],
              [0.2, 0.5, 0.2, 0.1]])
print(cross_entropy_error(y, t))

# 레이블 형식
t = np.array([2, 1])  # 정답 인덱스
y = np.array([[0.1, 0.2, 0.6, 0.1],
              [0.2, 0.5, 0.2, 0.1]])
print(cross_entropy_error(y, t))
```

---

## 4. 왜 손실 함수를 사용하는가? ⭐⭐⭐

### 정확도를 지표로 사용하면 안 되는 이유

```python
"""
정확도 문제점:
1. 불연속적 (0% → 100%)
2. 미분이 대부분 0
3. 매개변수의 미세한 변화를 반영 못함

예:
가중치를 조금 변경 → 정확도 변화 없음 (33% → 33%)
→ 기울기가 0 → 학습 불가능

손실 함수:
1. 연속적
2. 미분 가능
3. 매개변수의 미세한 변화 반영
→ 학습 가능!
"""
```

**예시:**
```python
# 100개 데이터 중 32개 맞춤 → 정확도 32%
# 가중치 조금 변경
# 100개 데이터 중 32개 맞춤 → 정확도 32% (변화 없음!)

# 하지만 손실 함수는:
# 손실: 2.3
# 가중치 조금 변경
# 손실: 2.29 (감소!)
```

---

## 5. 수치 미분 ⭐⭐

### 미분의 정의

**해석적 미분:**
$$\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x+h) - f(x)}{h}$$

**문제점:** 반올림 오차 (rounding error)

### 수치 미분 (Numerical Differentiation)

**중심 차분 / 중앙 차분:**
$$\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x+h) - f(x-h)}{2h}$$

**구현:**
```python
def numerical_diff(f, x):
    """수치 미분"""
    h = 1e-4  # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)

# 예시: f(x) = x^2
def function_1(x):
    return x**2

# x=5에서의 미분
print(numerical_diff(function_1, 5))   # 10.000000000139778
print(numerical_diff(function_1, 10))  # 20.00000000279556

# 해석적 미분: f'(x) = 2x
# x=5 → 10
# x=10 → 20
```

---

## 6. 편미분 ⭐⭐

### 다변수 함수의 미분

**함수:**
$$f(x_0, x_1) = x_0^2 + x_1^2$$

**구현:**
```python
def function_2(x):
    """f(x0, x1) = x0^2 + x1^2"""
    return x[0]**2 + x[1]**2
    # 또는 return np.sum(x**2)
```

### 편미분 계산

**문제:** $x_0 = 3, x_1 = 4$일 때, $\frac{\partial f}{\partial x_0}$를 구하라.

```python
# x1을 4로 고정
def function_tmp1(x0):
    return x0**2 + 4.0**2

print(numerical_diff(function_tmp1, 3.0))  # 6.00000000000378

# 해석적 미분: ∂f/∂x0 = 2x0
# x0=3 → 2*3 = 6
```

**문제:** $x_0 = 3, x_1 = 4$일 때, $\frac{\partial f}{\partial x_1}$를 구하라.

```python
# x0을 3으로 고정
def function_tmp2(x1):
    return 3.0**2 + x1**2

print(numerical_diff(function_tmp2, 4.0))  # 7.999999999999119

# 해석적 미분: ∂f/∂x1 = 2x1
# x1=4 → 2*4 = 8
```

---

## 7. 기울기 (Gradient) ⭐⭐⭐

### 기울기란?

**모든 변수의 편미분을 벡터로 정리한 것**

$$\nabla f = \left(\frac{\partial f}{\partial x_0}, \frac{\partial f}{\partial x_1}\right)$$

### 구현

```python
def numerical_gradient(f, x):
    """기울기 계산"""
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)  # x와 같은 형상의 배열
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        # 중심 차분
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        # 값 복원
        x[idx] = tmp_val
    
    return grad

# 테스트
def function_2(x):
    return x[0]**2 + x[1]**2

# (3, 4)에서의 기울기
print(numerical_gradient(function_2, np.array([3.0, 4.0])))
# [6. 8.]

# (0, 2)에서의 기울기
print(numerical_gradient(function_2, np.array([0.0, 2.0])))
# [0. 4.]

# (3, 0)에서의 기울기
print(numerical_gradient(function_2, np.array([3.0, 0.0])))
# [6. 0.]
```

### 기울기의 의미

**기울기가 가리키는 방향 = 각 지점에서 함수의 출력 값을 가장 크게 줄이는 방향**

```python
"""
f(x0, x1) = x0^2 + x1^2

(3, 4)에서 기울기: [6, 8]
→ x0를 음의 방향으로, x1을 음의 방향으로 이동하면
  함수 값이 감소

원점 (0, 0)에서 기울기: [0, 0]
→ 최솟값 지점 (극솟값)
"""
```

---

## 8. 경사하강법 (Gradient Descent) ⭐⭐⭐

### 개념

**기울기를 이용하여 함수의 최솟값을 찾는 방법**

$$x_{new} = x_{old} - \eta \nabla f(x_{old})$$

- $\eta$: 학습률 (learning rate)
- $\nabla f$: 기울기

### 구현

```python
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    """
    경사하강법
    
    f: 최적화할 함수
    init_x: 초깃값
    lr: 학습률 (learning rate)
    step_num: 반복 횟수
    """
    x = init_x
    x_history = []  # 이동 경로 기록
    
    for i in range(step_num):
        x_history.append(x.copy())
        
        # 기울기 계산
        grad = numerical_gradient(f, x)
        
        # 갱신
        x -= lr * grad
    
    return x, np.array(x_history)

# 테스트
def function_2(x):
    return x[0]**2 + x[1]**2

# 초깃값 (-3, 4)
init_x = np.array([-3.0, 4.0])

# 학습률 0.1
x, history = gradient_descent(function_2, init_x, lr=0.1, step_num=100)
print(x)  # [-6.11110793e-10  8.14814391e-10] ≈ (0, 0)
```

### 학습률의 중요성 ⭐⭐

```python
# 학습률이 너무 큰 경우
x, _ = gradient_descent(function_2, init_x, lr=10.0, step_num=100)
print(x)  # [-2.58983747e+13 -1.29524862e+12] → 발산!

# 학습률이 너무 작은 경우
x, _ = gradient_descent(function_2, init_x, lr=1e-10, step_num=100)
print(x)  # [-2.99999994  3.99999992] → 거의 이동 안 함!

# 적절한 학습률
x, _ = gradient_descent(function_2, init_x, lr=0.1, step_num=100)
print(x)  # [0. 0.] → 최솟값 도달!
```

**핵심:**
- 학습률이 너무 크면 → 발산
- 학습률이 너무 작으면 → 학습 안 됨
- 적절한 학습률 선택이 중요 (하이퍼파라미터)

---

## 9. 신경망의 기울기 ⭐⭐⭐

### SimpleNet 구현

```python
import numpy as np
import sys, os
sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class SimpleNet:
    def __init__(self):
        # 가중치 초기화 (2×3 행렬)
        self.W = np.random.randn(2, 3)
    
    def predict(self, x):
        """예측"""
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        """손실 계산"""
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

# 사용 예시
net = SimpleNet()
print(net.W)
# [[ 0.47355232  0.9977393   0.84668094]
#  [ 0.85557411  0.03563661  0.69422093]]

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
# [1.05414809 0.63071653 1.13269074]

print(np.argmax(p))  # 2 (최댓값 인덱스)

t = np.array([0, 0, 1])  # 정답 레이블
print(net.loss(x, t))    # 0.92806853663411326
```

### 가중치에 대한 기울기

```python
def f(W):
    """손실 함수 (가중치를 인수로)"""
    return net.loss(x, t)

# 기울기 계산
dW = numerical_gradient(f, net.W)
print(dW)
# [[ 0.21924763  0.14356247 -0.36281009]
#  [ 0.32887144  0.2153437  -0.54421514]]
```

### 기울기의 의미

```python
"""
dW[0, 0] ≈ 0.22
→ w11을 h만큼 증가시키면 손실이 0.22h만큼 증가

dW[1, 2] ≈ -0.54
→ w23을 h만큼 증가시키면 손실이 0.54h만큼 감소

따라서:
- 양수 기울기: 가중치를 감소시켜야 함
- 음수 기울기: 가중치를 증가시켜야 함
- 절댓값이 클수록: 손실에 더 큰 영향
"""
```

---

## 10. 학습 알고리즘 구현 ⭐⭐⭐

### 신경망 학습 4단계

```python
"""
1단계: 미니배치
   - 훈련 데이터 중 일부를 무작위로 선택

2단계: 기울기 산출
   - 각 가중치 매개변수에 대한 손실 함수의 기울기 계산

3단계: 매개변수 갱신
   - 가중치를 기울기 방향으로 조금 갱신

4단계: 반복
   - 1~3단계를 반복
"""
```

### 2층 신경망 클래스

```python
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, 
                 weight_init_std=0.01):
        """
        초기화
        
        input_size: 입력층 뉴런 수
        hidden_size: 은닉층 뉴런 수
        output_size: 출력층 뉴런 수
        """
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                           np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                           np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        """예측 (순전파)"""
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        """손실 함수 값"""
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        """정확도"""
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        """기울기 계산 (수치 미분)"""
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
```

### 학습 구현

```python
from dataset.mnist import load_mnist

# 데이터 로드
(x_train, t_train), (x_test, t_test) = load_mnist(
    normalize=True, 
    one_hot_label=True
)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

# 신경망 생성
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, x_test)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(f"train acc, test acc | {train_acc}, {test_acc}")
```

---

## 11. 용어 정리 ⭐⭐⭐

### 에폭 (Epoch)

**모든 훈련 데이터를 한 번씩 학습한 것**

```python
"""
훈련 데이터: 60,000개
미니배치: 100개

1 에폭 = 60,000 / 100 = 600회 반복
"""
```

### 확률적 경사하강법 (SGD: Stochastic Gradient Descent)

**무작위로 선택한 미니배치에 대해 경사하강법을 수행**

```python
"""
전체 데이터: 비용이 너무 큼
미니배치: 무작위 선택 → 확률적 (Stochastic)

장점:
- 빠른 학습
- 메모리 효율적
- 지역 최솟값 탈출 가능
"""
```

---

## 12. 시험 예상 문제

### 문제 1 (10점)
```
오차제곱합(SSE)과 교차 엔트로피 오차(CEE)의 수식을 쓰고,
각각 어떤 문제에 주로 사용되는지 설명하시오.
```

**답안:**
```
1. 오차제곱합 (SSE):
   E = (1/2)Σ(yk - tk)²
   - 회귀 문제에 주로 사용
   - 예측값과 실제값의 차이를 제곱하여 합산

2. 교차 엔트로피 오차 (CEE):
   E = -Σtk log(yk)
   - 분류 문제에 주로 사용
   - 정답 레이블에 해당하는 출력의 로그를 계산
   - 확률 분포 간의 차이를 측정
```

---

### 문제 2 (10점)
```
신경망 학습에서 정확도가 아닌 손실 함수를 사용하는 
이유를 설명하시오.
```

**답안:**
```
정확도는 불연속적이고 대부분의 지점에서 미분값이 0이 되어
매개변수의 미세한 변화를 반영하지 못하기 때문이다.

예를 들어, 100개 데이터 중 32개를 맞추면 정확도는 32%이고,
가중치를 조금 변경해도 여전히 32개를 맞추면 정확도는 32%로
변화가 없다. 이 경우 기울기가 0이 되어 학습이 불가능하다.

반면 손실 함수는 연속적이고 미분 가능하여 매개변수의
미세한 변화도 반영할 수 있으므로 학습이 가능하다.
```

---

### 문제 3 (15점)
```
다음 함수에 대해 (3, 4) 지점에서의 기울기를 구하시오.

f(x0, x1) = x0² + x1²

그리고 기울기의 의미를 설명하시오.
```

**답안:**
```python
# 편미분
∂f/∂x0 = 2x0 = 2(3) = 6
∂f/∂x1 = 2x1 = 2(4) = 8

# 기울기
∇f = (6, 8)

# 의미:
기울기 (6, 8)은 (3, 4) 지점에서 함수 f의 값이
가장 가파르게 증가하는 방향을 나타낸다.

따라서 함수의 최솟값을 찾으려면 기울기의 반대 방향인
(-6, -8) 방향으로 이동해야 한다.

이것이 경사하강법의 원리이다.
```

---

### 문제 4 (15점)
```
경사하강법의 갱신 수식을 쓰고, 학습률이 너무 크거나
너무 작을 때 발생하는 문제를 설명하시오.
```

**답안:**
```
갱신 수식:
x_new = x_old - η∇f(x_old)

여기서 η는 학습률(learning rate)

학습률이 너무 큰 경우:
- 갱신 폭이 너무 커서 최솟값을 지나쳐 버림
- 발산(divergence)할 수 있음
- 예: η = 10.0 → 값이 무한대로 증가

학습률이 너무 작은 경우:
- 갱신 폭이 너무 작아서 학습이 느림
- 최솟값에 도달하지 못함
- 예: η = 1e-10 → 거의 이동하지 않음

따라서 적절한 학습률을 선택하는 것이 중요하다.
(일반적으로 0.01, 0.001 등 사용)
```

---

### 문제 5 (10점)
```
미니배치 학습의 장점 3가지를 설명하시오.
```

**답안:**
```
1. 계산 효율성
   - 전체 데이터를 한 번에 처리하는 것보다 빠름
   - 행렬 연산의 최적화 활용

2. 메모리 효율성
   - 전체 데이터를 메모리에 올릴 필요 없음
   - GPU 메모리 제약 극복

3. 일반화 성능 향상
   - 무작위 선택으로 인한 노이즈가 
     지역 최솟값 탈출에 도움
   - 과적합(overfitting) 방지
```

---

## 13. 핵심 개념 정리

### ✅ 반드시 알아야 할 것

1. **손실 함수**
   - SSE: 회귀 문제
   - CEE: 분류 문제
   - 원-핫 인코딩

2. **미니배치**
   - 일부 데이터로 학습
   - 평균 손실 계산
   - 확률적 경사하강법

3. **수치 미분**
   - 중심 차분 사용
   - h = 1e-4
   - 근사값 계산

4. **기울기**
   - 모든 변수의 편미분
   - 함수가 가장 크게 감소하는 방향
   - 경사하강법에 사용

5. **경사하강법**
   - x_new = x_old - η∇f
   - 학습률의 중요성
   - 반복적 갱신

6. **학습 알고리즘**
   - 미니배치 → 기울기 → 갱신 → 반복
   - 에폭: 전체 데이터 1회 학습
   - SGD: 확률적 경사하강법

---

## 14. 빠른 복습 체크리스트

- [ ] 손실 함수의 개념과 종류
- [ ] SSE와 CEE 수식
- [ ] 미니배치 학습
- [ ] 정확도 대신 손실 함수를 쓰는 이유
- [ ] 수치 미분 (중심 차분)
- [ ] 편미분의 개념
- [ ] 기울기의 의미
- [ ] 경사하강법 수식
- [ ] 학습률의 역할
- [ ] 신경망 학습 4단계
- [ ] 에폭과 SGD 개념

---

## 15. 코드 템플릿 (암기용)

### 손실 함수

```python
# SSE
def sum_squares_error(y, t):
    return 0.5 * np.sum((y - t)**2)

# CEE
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))
```

### 수치 미분

```python
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)
```

### 기울기

```python
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
    
    return grad
```

### 경사하강법

```python
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x
```

**시험 화이팅! 🎯**